# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": nba (football, in [assets/nba.csv](assets/nba.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def clear_data(string):
    return re.sub(r'\[[a-z]* [0-9]+\]', '', string)


def clear_nba_data(string):
    if re.search(r'\*|\+', string) is None:
        return string
    else:
        return re.sub(r'\*|\+', '', string)


def get_area(team,nfl_cities):
    for index, row in nfl_cities.iterrows():
        if team in index:
            return row['Metropolitan area']



def get_nfl_data():
    return outdf


def nhl_correlation():

    population_by_region = []
    win_loss_by_region = []

    nfldf= pd.read_csv("assets/nfl.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]


    nfldf= nfldf[nfldf['year'] == 2018].drop([0, 5, 10, 15, 20, 25, 30, 35])

    population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
    population = population.set_index('Metropolitan area')
    #print(population)
    cities['NFL'] = cities['NFL'].apply(lambda x: clear_data(x))
    nfl_cities = cities[['Metropolitan area', 'NFL']].set_index('NFL')
    nfl_cities = nfl_cities.drop(['—', ''], axis=0)

    nfldf['team'] = nfldf['team'].apply(lambda x: clear_nba_data(x))
 

    nfldf['area'] = nfldf['team'].apply(lambda x: x.split(" ")[-1])
    nfldf['area'] = nfldf['area'].apply(lambda x: get_area(x,nfl_cities))

    out = []
    #print(nfldf)
    for group, frame in nfldf.groupby('area'):
        total_wins = frame['W'].astype(int).sum()
        total_losses = frame['L'].astype(int).sum()
        total_matches = total_wins + total_losses
        ratio = total_wins / total_matches
        out_dict = {
            'Area': group,
            'Ratio': ratio
        }
        out.append(out_dict)

    newdf= pd.DataFrame(out)
    newdf= newdf.set_index('Area')

    outdf= pd.merge(newdf, population, how="inner", left_index=True, right_index=True)
    outdf['Population (2016 est.)[8]'] = pd.to_numeric(outdf['Population (2016 est.)[8]'])

    population_by_region = outdf['Population (2016 est.)[8]'].tolist()
    win_loss_by_region = outdf['Ratio'].tolist()

    corr = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    return corr

nhl_correlation()

0.00492211214934941

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def clear_data(string):
    return re.sub(r'\[[a-z]* [0-9]+\]', '', string)


def clear_nba_data(string):
    result= re.sub(r'\s*\*?\s*\(\d+\)$', '', string)
    return result


def get_area(team,nfl_cities):
    for index, row in nfl_cities.iterrows():
        if team in index:
            return row['Metropolitan area']


def get_nba_data():
    return outdf

def nba_correlation():
    population_by_region = []  # pass in metropolitan area population from cities
    win_loss_by_region = []  # pass in win/loss ratio from nhldfin the same order as cities["Metropolitan area"]
    nbadf= pd.read_csv("assets/nba.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    

    cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

    nbadf= nbadf[nbadf['year'] == 2018]  # get only 2018 stats no need of dropping rows
    population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]

    #print(population)
    population = population.set_index('Metropolitan area')
    cities['NBA'] = cities['NBA'].apply(lambda x: clear_data(x))
    #print(cities['NBA'])
    nba_cities = cities[['Metropolitan area', 'NBA']].set_index('NBA')

    nba_cities = nba_cities.drop(['—', ''], axis=0)
    #print(nba_cities)
   
    
    nbadf['team'] = nbadf['team'].apply(lambda x: clear_nba_data(x))
    
    nbadf['area'] = nbadf['team'].apply(lambda x: x.split(" ")[-1])

    nbadf['area'] = nbadf['area'].apply(lambda x: get_area(x,nba_cities))

    #nbadf['area']=cities['Metropolitan area']
    print(nbadf)
    #print(population)
    out = []
    for group, frame in nbadf.groupby('area'):
        total_wins = np.sum(pd.to_numeric(frame['W']))
        total_losses = np.sum(pd.to_numeric(frame['L']))
        WL=np.sum(pd.to_numeric(frame['W/L%']))
        total_matches = total_wins + total_losses
        ratio = (total_wins / total_matches)
        out_dict = {
            'Area': group,
            'Ratio': ratio
        }
        out.append(out_dict)
    newdf= pd.DataFrame(out)
    newdf= newdf.set_index('Area')
    #print(newdf)
    #print(population)
    outdf= pd.merge(newdf, population, how="inner", left_index=True, right_index=True)
    outdf['Population (2016 est.)[8]'] = pd.to_numeric(outdf['Population (2016 est.)[8]'])
    population_by_region = outdf['Population (2016 est.)[8]'].tolist()
    win_loss_by_region = outdf['Ratio'].to_list()
    corr = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    return corr

nba_correlation()

                      team   W   L                 W/L%    GB   PS/G   PA/G  \
0          Toronto Raptors  59  23                 0.72     —  111.7  103.9   
1           Boston Celtics  55  27   0.6709999999999999   4.0  104.0  100.4   
2       Philadelphia 76ers  52  30                0.634   7.0  109.8  105.3   
3      Cleveland Cavaliers  50  32                 0.61   9.0  110.9  109.9   
4           Indiana Pacers  48  34                0.585  11.0  105.6  104.2   
5               Miami Heat  44  38                0.537  15.0  103.4  102.9   
6          Milwaukee Bucks  44  38                0.537  15.0  106.5  106.8   
7       Washington Wizards  43  39                0.524  16.0  106.6  106.0   
8          Detroit Pistons  39  43  0.47600000000000003  20.0  103.8  103.9   
9        Charlotte Hornets  36  46                0.439  23.0  108.2  108.0   
10         New York Knicks  29  53                0.354  30.0  104.5  108.0   
11           Brooklyn Nets  28  54                0.

-0.17657160252844614

In [8]:
import re

string1 = "Toronto Raptors* (1)"
string2 = "Orlando Magic (14)"

regex_pattern = r'\s*\*?\s*\(\d+\)$'
result1 = re.sub(regex_pattern, '', string1)
result2 = re.sub(regex_pattern, '', string2)

print(result1)  # Output: "Toronto Raptors"
print(result2)  # Output: "Orlando Magic"


Toronto Raptors
Orlando Magic


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [11]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def mlb_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    cities.drop([24,25,26,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,48,49,50],0,inplace=True)
    
    mlb_df = mlb_df.head(30)
    
    mlb_df['team_ville'] = mlb_df['team']
    mlb_df['team_ville'] = mlb_df['team_ville'].map({'Boston Red Sox':'Boston',
     'New York Yankees':'New York City',
     'Tampa Bay Rays':'Tampa Bay Area',
     'Toronto Blue Jays':'Toronto',
     'Baltimore Orioles':'Baltimore',
     'Cleveland Indians':'Cleveland',
     'Minnesota Twins':'Minneapolis–Saint Paul',
     'Detroit Tigers':'Detroit',
     'Chicago White Sox':'Chicago',
     'Kansas City Royals':'Kansas City',
     'Houston Astros':'Houston',
     'Oakland Athletics':'San Francisco Bay Area',
     'Seattle Mariners':'Seattle',
     'Los Angeles Angels':'Los Angeles',
     'Texas Rangers':'Dallas–Fort Worth',
     'Atlanta Braves':'Atlanta',
     'Washington Nationals':'Washington, D.C.',
     'Philadelphia Phillies':'Philadelphia',
     'New York Mets':'New York City',
     'Miami Marlins':'Miami–Fort Lauderdale',
     'Milwaukee Brewers':'Milwaukee',
     'Chicago Cubs':'Chicago',
     'St. Louis Cardinals':'St. Louis',
     'Pittsburgh Pirates':'Pittsburgh',
     'Cincinnati Reds':'Cincinnati',
     'Los Angeles Dodgers':'Los Angeles',
     'Colorado Rockies':'Denver',
     'Arizona Diamondbacks':'Phoenix',
     'San Francisco Giants':'San Francisco Bay Area',
     'San Diego Padres':'San Diego'})
    
    df3 = pd.merge(mlb_df,cities, left_on= "team_ville", right_on= "Metropolitan area")
    
    #df2['W/L%'] = pd.to_numeric(df2['W/L%'])
    df3['W'] = pd.to_numeric(df3['W'])
    df3['L'] = pd.to_numeric(df3['L'])
    df3['Population (2016 est.)[8]'] = pd.to_numeric(df3['Population (2016 est.)[8]'])
    he = ['team','W','L','Metropolitan area','Population (2016 est.)[8]']
    df3 = df3[he]
    df3['W/L'] = df3['W']/(df3['L']+df3['W'])
    df3 = df3.groupby('Metropolitan area').mean().reset_index()
    
    
    population_by_region = df3['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df3['W/L'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()

C:\Users\Pierre\AppData\Local\Temp\ipykernel_14556\3015127102.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  cities.drop([24,25,26,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,48,49,50],0,inplace=True)


0.15027698302669307

In [199]:
import re

input_string = "DodgersAngels"

# Remplacer "Nets" par "\nNets", "Angels" par "\nAngels", etc.
output_string = re.sub(r'([a-zA-Z]+)([A-Z][a-z]+)', r'\1\n\2', input_string)

# Supprimer les balises de note [note 10], [note 12], [note 14]
output_string = re.sub(r'\[note \d+\]', '', output_string)

print(output_string)


Dodgers
Angels


In [2]:
import re
import pandas as pd

# Fonction clear_data modifiée pour retourner deux valeurs
def clear_data(string1):
    # Remplacer "Nets" par "\nNets", "Angels" par "\nAngels", etc.
    output_string = re.sub(r'([a-zA-Z]+)([A-Z][a-z]+)', r'\1\n\2', input_string)

    # Supprimer les balises de note [note 10], [note 12], [note 14]
    output_string = re.sub(r'\[note \d+\]', '', output_string)
    return output_string


# Exemple de données pour cities
data = {'Metropolitan area': ['City A', 'City B', 'City C'],
        'MLB': ['Team A', 'DodgersAngels[note 10]', 'Team C [note 12]']}
cities = pd.DataFrame(data)


print(cities)
# Appliquer la fonction clear_data avec gestion des deux valeurs retournées
cities['MLB'] = cities['MLB'].apply(lambda x: [clear_data(x), x.split(" ")[-1]][-1])

# Afficher le résultat
print(cities)



  Metropolitan area                     MLB
0            City A                  Team A
1            City B  DodgersAngels[note 10]
2            City C        Team C [note 12]


NameError: name 'input_string' is not defined

In [1]:
import pandas as pd
import re

# Fonction clear_data modifiée pour retourner une liste de valeurs distinctes
def clear_data(string1):
    # Remplacer "Nets" par "\nNets", "Angels" par "\nAngels", etc.
    output_string = re.sub(r'([a-zA-Z]+)([A-Z][a-z]+)', r'\1,\2', string1)

    # Supprimer les balises de note [note 10], [note 12], [note 14]
    output_string = re.sub(r'\[note \d+\]', '', output_string)
    
    # Convertir la chaîne en liste
    output_list = output_string.split(',')

    return output_list

# Exemple de données pour cities
data = {'Metropolitan area': ['City A', 'City B', 'City C','Cityd'],
        'MLB': ['Tea','DodgersAngels[note 10]', 'Team [note 12]','KnicksNets']}
cities = pd.DataFrame(data)

# Appliquer la fonction clear_data et étendre le DataFrame avec les valeurs distinctes
new_rows = []
for index, row in cities.iterrows():
    cleaned_values = clear_data(row['MLB'])
    if len(cleaned_values) > 1:
        for value in cleaned_values[1:]:
            new_rows.append({'Metropolitan area': row['Metropolitan area'], 'MLB': value})
    row['MLB'] = cleaned_values[0]

# Étendre le DataFrame avec les nouvelles lignes
cities = cities.append(new_rows, ignore_index=True)

# Afficher le résultat
print(cities)
#print(cities.loc[cities['Metropolitan area']=='City A','MLB'][0])
valeurs_correspondantes = cities.loc[cities['MLB'].str.contains('Dorrrrrrrrd'), 'Metropolitan area'].iloc[0]
print(valeurs_correspondantes)

C:\Users\Pierre\AppData\Local\Temp\ipykernel_3912\1891125225.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cities = cities.append(new_rows, ignore_index=True)


  Metropolitan area      MLB
0            City A      Tea
1            City B  Dodgers
2            City C    Team 
3             Cityd   Knicks
4            City B   Angels
5             Cityd     Nets


IndexError: single positional indexer is out-of-bounds

In [1]:
import pandas as pd
import re

population_by_region = []  # pass in metropolitan area population from cities
win_loss_by_region = []  # pass in win/loss ratio from nhldfin the same order as cities["Metropolitan area"]
mlbdf= pd.read_csv("assets/mlb.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
    #print(cities)  

cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
cities.rename(columns={"Population (2016 est.)[8]": "Population"},
              inplace=True)
mlbdf= mlbdf[mlbdf['year'] == 2018]  # get only 2018 stats no need of dropping rows
population = cities[['Metropolitan area', 'Population']]

print(mlbdf.loc[mlbdf['team'].str.contains('White'),'W-L%'].iloc[0])

0.383


In [146]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
cities.rename(columns={"Population (2016 est.)[8]": "Population"},inplace=True)

def clear_data(string1):
    # Remplacer "Nets" par "\nNets", "Angels" par "\nAngels", etc.
    output_string = re.sub(r'([a-zA-Z]+)([A-Z][a-z]+)', r'\1,\2', string1)

    # Supprimer les balises de note [note 10], [note 12], [note 14]
    output_string = re.sub(r'\[note \d+\]', '', output_string)

    utput_string = re.sub(r'[^a-zA-Z\s]+', '', output_string)
    
    # Convertir la chaîne en liste
    output_list = output_string.split(',')

    return output_list


def get_area(team,mlb_cities):
    for index, row in mlb_cities.iterrows():
        if team in index:
            return row['Metropolitan area']

def mlb_correlation():
    #print(cities['MLB'])
    team = cities['MLB'].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    
    
    team['Metropolitan area'] = cities['Metropolitan area']
    

    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("", np.nan).replace("—", np.nan).dropna().reset_index().rename(columns={"value": "team"})
    team = pd.merge(team, cities, how='left', on='Metropolitan area').iloc[:, 1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team'] = team['team'].str.replace('\ Sox', 'Sox')
    team['team'] = team['team'].str.replace('[\w.]*\ ', '')

    df= pd.read_csv("assets/mlb.csv")
    df= df[df['year'] == 2018]
    df['team'] = df['team'].str.replace(r'[\*]', "")
    df['team'] = df['team'].str.replace(r'\(\d*\)', "")
    df['team'] = df['team'].str.replace(r'[\xa0]', "")
    df= df[['team', 'W-L%']]

    #df.rename(columns={"W-L%": "W/L%"}, inplace=True)
    df['team'] = df['team'].str.replace('\ Sox', 'Sox')
    df['team'] = df['team'].str.replace('[\w.]* ', '')
    df= df.astype({'team': str, 'W-L%': float})
    df=df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
    merge = pd.merge(team, df, 'outer', on='team')
    
    merge = merge.groupby('Metropolitan area').agg({'W-L%': np.nanmean, 'Population': np.nanmean})
    merge=merge.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
    population_by_region = merge['Population']  # pass in metropolitan area population from cities
    win_loss_by_region = merge['W-L%']  # pass in win/loss ratio from dfin the same order as cities["Metropolitan area"]
    print(merge.iloc[1:20])

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()




                         W-L%  Population
Metropolitan area                        
Baltimore               0.290   2798886.0
Boston                  0.667   4794447.0
Chicago                 0.483   9512999.0
Cincinnati              0.414   2165139.0
Cleveland               0.562   2055612.0
Dallas–Fort Worth       0.414   7233323.0
Denver                  0.558   2853077.0
Detroit                 0.395   4297617.0
Houston                 0.636   6772470.0
Kansas City             0.358   2104509.0
Los Angeles             0.529  13310447.0
Miami–Fort Lauderdale   0.391   6066387.0
Milwaukee               0.589   1572482.0
Minneapolis–Saint Paul  0.481   3551036.0
New York City           0.546  20153634.0
Philadelphia            0.494   6070500.0
Phoenix                 0.506   4661537.0
Pittsburgh              0.509   2342299.0
San Diego               0.407   3317749.0


C:\Users\Pierre\AppData\Local\Temp\ipykernel_3912\3231281658.py:41: FutureWarning: The default value of regex will change from True to False in a future version.
  team['team'] = team['team'].str.replace('\ Sox', 'Sox')
C:\Users\Pierre\AppData\Local\Temp\ipykernel_3912\3231281658.py:42: FutureWarning: The default value of regex will change from True to False in a future version.
  team['team'] = team['team'].str.replace('[\w.]*\ ', '')
C:\Users\Pierre\AppData\Local\Temp\ipykernel_3912\3231281658.py:46: FutureWarning: The default value of regex will change from True to False in a future version.
  df['team'] = df['team'].str.replace(r'[\*]', "")
C:\Users\Pierre\AppData\Local\Temp\ipykernel_3912\3231281658.py:47: FutureWarning: The default value of regex will change from True to False in a future version.
  df['team'] = df['team'].str.replace(r'\(\d*\)', "")
C:\Users\Pierre\AppData\Local\Temp\ipykernel_3912\3231281658.py:48: FutureWarning: The default value of regex will change from True

0.15003737475409495

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **nba** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nbadf=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nbadfin the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for nba"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
cities.rename(columns={"Population (2016 est.)[8]": "Population"},
              inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")


def nfl_correlation(): 
    team = cities['NFL'].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
    team=pd.merge(team,cities,how='left',on = 'Metropolitan area').iloc[:,1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('[\w.]*\ ','')
    
    df=pd.read_csv("assets/NFL.csv")
    df= df[df['year'] == 2018]
    df['team'] = df['team'].str.replace(r'[\*]',"")
    df['team'] = df['team'].str.replace(r'\(\d*\)',"")
    df['team'] = df['team'].str.replace(r'[\xa0]',"")
    df= df[['team','W-L%']]
    df.rename(columns={"W-L%": "W/L%"},inplace=True)
    dropList=[]
    for i in range(df.shape[0]):
        row=df.iloc[i]
        if row['team']==row['W/L%']:
            dropList.append(i)
    df=df.drop(dropList)

    df['team'] = df['team'].str.replace('[\w.]* ','')
    df['team'] = df['team'].str.replace('+','')
    df= df.astype({'team': str,'W/L%': float})
    
    merge=pd.merge(team,df,'outer', on = 'team')
    merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

    population_by_region = merge['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = merge['W/L%'] # pass in win/loss ratio from dfin the same order as cities["Metropolitan area"]   
    
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

C:\Users\Pierre\AppData\Local\Temp\ipykernel_14556\751409186.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
C:\Users\Pierre\AppData\Local\Temp\ipykernel_14556\751409186.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  team['team']=team['team'].str.replace('[\w.]*\ ','')
C:\Users\Pierre\AppData\Local\Temp\ipykernel_14556\751409186.py:23: FutureWarning: The default value of regex will change from True to False in a future version.
  df['team'] = df['team'].str.replace(r'[\*]',"")
C:\Users\Pierre\AppData\Local\Temp\ipykernel_14556\751409186.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
  df['team'] = df['team'].str.replace(r'\(\d*\)',"")
C:\Users\Pierre\AppData\Local\Temp\ipykernel_14556\751409186.py:25: FutureWarning: The default value of regex will change from True to 

0.0042821414363930456

In [6]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nfl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()

    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nfl_df.drop([0,5,10,15,20,25,30,35],0,inplace=True)
    print(cities)

    cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
    team= cities['NFL'].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team.to_li

    cities.drop([13,22,27,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,49,50],0,inplace=True)
    
    l1 = []
    for i in nfl_df['team']:
        #i=i.rstrip()
        i=i.split('*')
        l1.append(i[0])
    nfl_df['team'] = l1
    
    l2 = []
    for i in nfl_df['team']:
        i=i.split('+')
        l2.append(i[0])
    nfl_df['team'] = l2
    
    nfl_df = nfl_df.head(32)
    
    nfl_df['team_ville'] = nfl_df['team']
    nfl_df['team_ville'] = nfl_df['team_ville'].map({'New England Patriots':'Boston',
     'Miami Dolphins':'Miami–Fort Lauderdale',
     'Buffalo Bills':'Buffalo',
     'New York Jets':'New York City',
     'Baltimore Ravens':'Baltimore',
     'Pittsburgh Steelers':'Pittsburgh',
     'Cleveland Browns':'Cleveland',
     'Cincinnati Bengals':'Cincinnati',
     'Houston Texans':'Houston',
     'Indianapolis Colts':'Indianapolis',
     'Tennessee Titans':'Nashville',
     'Jacksonville Jaguars':'Jacksonville',
     'Kansas City Chiefs':'Kansas City',
     'Los Angeles Chargers':'Los Angeles',
     'Denver Broncos':'Denver',
     'Oakland Raiders':'San Francisco Bay Area',
     'Dallas Cowboys':'Dallas–Fort Worth',
     'Philadelphia Eagles':'Philadelphia',
     'Washington Redskins':'Washington, D.C.',
     'New York Giants':'New York City',
     'Chicago Bears':'Chicago',
     'Minnesota Vikings':'Minneapolis–Saint Paul',
     'Green Bay Packers':'Green Bay',
     'Detroit Lions':'Detroit',
     'New Orleans Saints':'New Orleans',
     'Carolina Panthers':'Charlotte',
     'Atlanta Falcons':'Atlanta',
     'Tampa Bay Buccaneers':'Tampa Bay Area',
     'Los Angeles Rams':'Los Angeles',
     'Seattle Seahawks':'Seattle',
     'San Francisco 49ers':'San Francisco Bay Area',
     'Arizona Cardinals':'Phoenix'}) 
    
    df4 = pd.merge(nfl_df,cities, left_on= "team_ville", right_on= "Metropolitan area")
    
    
    df4['W'] = pd.to_numeric(df4['W'])
    df4['L'] = pd.to_numeric(df4['L'])
    df4['Population (2016 est.)[8]'] = pd.to_numeric(df4['Population (2016 est.)[8]'])
    he = ['team','W','L','Metropolitan area','Population (2016 est.)[8]']
    df4 = df4[he]
    df4['W/L'] = df4['W']/(df4['L']+df4['W'])
    df4 = df4.groupby('Metropolitan area').mean().reset_index()

    
    population_by_region = df4['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df4['W/L'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

         Metropolitan area Population (2016 est.)[8]                   NFL  \
0            New York City                  20153634    GiantsJets[note 1]   
1              Los Angeles                  13310447  RamsChargers[note 4]   
2   San Francisco Bay Area                   6657982  49ersRaiders[note 6]   
3                  Chicago                   9512999         Bears[note 8]   
4        Dallas–Fort Worth                   7233323               Cowboys   
5         Washington, D.C.                   6131977              Redskins   
6             Philadelphia                   6070500                Eagles   
7                   Boston                   4794447     Patriots[note 14]   
8   Minneapolis–Saint Paul                   3551036               Vikings   
9                   Denver                   2853077               Broncos   
10   Miami–Fort Lauderdale                   6066387              Dolphins   
11                 Phoenix                   4661537            

C:\Users\Pierre\AppData\Local\Temp\ipykernel_14556\4053558280.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  nfl_df.drop([0,5,10,15,20,25,30,35],0,inplace=True)
C:\Users\Pierre\AppData\Local\Temp\ipykernel_14556\4053558280.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")


AttributeError: 'DataFrame' object has no attribute 'to_li'

In [10]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
cities.rename(columns={"Population (2016 est.)[8]": "Population"},
              inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")
cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")

Big4='NFL'

def nfl_correlation(): 
    team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
    team=pd.merge(team,cities,how='left',on = 'Metropolitan area').iloc[:,1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('[\w.]*\ ','')
    
    _df=pd.read_csv("assets/"+str.lower(Big4)+".csv")
    _df = _df[_df['year'] == 2018]
    _df['team'] = _df['team'].str.replace(r'[\*]',"")
    _df['team'] = _df['team'].str.replace(r'\(\d*\)',"")
    _df['team'] = _df['team'].str.replace(r'[\xa0]',"")
    _df = _df[['team','W-L%']]
    _df.rename(columns={"W-L%": "W/L%"},inplace=True)
    dropList=[]
    for i in range(_df.shape[0]):
        row=_df.iloc[i]
        if row['team']==row['W/L%']:
            dropList.append(i)
    _df=_df.drop(dropList)

    _df['team'] = _df['team'].str.replace('[\w.]* ','')
    _df['team'] = _df['team'].str.replace('+','')
    _df = _df.astype({'team': str,'W/L%': float})
    
    merge=pd.merge(team,_df,'outer', on = 'team')
    merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

    population_by_region = merge['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = merge['W/L%'] # pass in win/loss ratio from _df in the same order as cities["Metropolitan area"]   
    
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

C:\Users\Pierre\AppData\Local\Temp\ipykernel_14556\2689093289.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
C:\Users\Pierre\AppData\Local\Temp\ipykernel_14556\2689093289.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
C:\Users\Pierre\AppData\Local\Temp\ipykernel_14556\2689093289.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")
C:\Users\Pierre\AppData\Local\Temp\ipykernel_14556\2689093289.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")
C:\Users\Pierre\AppData\Local\Temp\ipykernel_14556\2689093289.py:23: FutureWarning: The default value of regex w

0.0042821414363930456

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlbdf=pd.read_csv("assets/mlb.csv")
nhldf=pd.read_csv("assets/nhl.csv")
nbadf=pd.read_csv("assets/nba.csv")
nbadf=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["nba","NBA"] should be the same as df.loc["NBA","nba"] and
    # df.loc["nba","nba"] should return np.nan
    sports = ['nba', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "nba"] - 0.80) <= 1e-2, "The MLB-nba p-value should be around 0.80"
    return p_values

In [9]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re



def nhl_correla(): 
    
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    nhl_df.drop([0,9,18,26],0,inplace=True)
    cities.drop([14,15,18,19,20,21,23,24,25,27,28,32,33,38,40,41,42,44,45,46,48,49,50],0,inplace=True)

    l= []
    for i in cities['NHL']:
        i=i.split('[')
        l.append(i[0])
    cities['NHL'] = l

    li = []
    for i in nhl_df['team']:
        i = re.findall("[^*]+", i)
        li.append(i[0])
    nhl_df['team'] = li

    nhl_df = nhl_df.head(31)

    nhl_df['team_ville'] = nhl_df['team']
    nhl_df['team_ville'] = nhl_df['team_ville'].map({'Tampa Bay Lightning':'Tampa Bay Area',
     'Boston Bruins':'Boston',
     'Toronto Maple Leafs':'Toronto',
     'Florida Panthers':'Miami–Fort Lauderdale',
     'Detroit Red Wings':'Detroit',
     'Montreal Canadiens':'Montreal',
     'Ottawa Senators':'Ottawa',
     'Buffalo Sabres':'Buffalo',
     'Washington Capitals':'Washington, D.C.',
     'Pittsburgh Penguins':'Pittsburgh',
     'Philadelphia Flyers':'Philadelphia',
     'Columbus Blue Jackets':'Columbus',
     'New Jersey Devils':'New York City',
     'Carolina Hurricanes':'Raleigh',
     'New York Islanders':'New York City',
     'New York Rangers':'New York City',
     'Nashville Predators':'Nashville',
     'Winnipeg Jets':'Winnipeg',
     'Minnesota Wild':'Minneapolis–Saint Paul',
     'Colorado Avalanche':'Denver',
     'St. Louis Blues':'St. Louis',
     'Dallas Stars':'Dallas–Fort Worth',
     'Chicago Blackhawks':'Chicago',
     'Vegas Golden Knights':'Las Vegas',
     'Anaheim Ducks':'Los Angeles',
     'San Jose Sharks':'San Francisco Bay Area',
     'Los Angeles Kings':'Los Angeles',
     'Calgary Flames':'Calgary',
     'Edmonton Oilers':'Edmonton',
     'Vancouver Canucks':'Vancouver',
     'Arizona Coyotes':'Phoenix'})

    df = pd.merge(nhl_df,cities, left_on= "team_ville", right_on= "Metropolitan area")

    df['W'] = pd.to_numeric(df['W'])
    df['L'] = pd.to_numeric(df['L'])
    df['Population (2016 est.)[8]'] = pd.to_numeric(df['Population (2016 est.)[8]'])

    he = ['team','W','L','Metropolitan area','Population (2016 est.)[8]']

    df = df[he]

    df['W/L'] = df['W']/(df['L']+df['W'])

    df = df.groupby('Metropolitan area').mean().reset_index()
    
    return df

def nba_correla():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    cities.drop([16,17,19,20,21,22,23,26,29,30,31,34,35,36,37,39,40,43,44,47,48,49,50],0,inplace=True)
    
    l1 = []
    for i in nba_df['team']:
        #i=i.rstrip()
        i=i.split('*')
        l1.append(i[0])
    nba_df['team'] = l1
    
    l2 = []
    for i in nba_df['team']:
        i=i.split('(')
        l2.append(i[0])
    nba_df['team'] = l2
    
    l3 = []
    for i in nba_df['team']:
        i=i.rstrip()
        l3.append(i)
    nba_df['team'] = l3
    

    nba_df = nba_df.head(30)

    nba_df['team_ville'] = nba_df['team']
    nba_df['team_ville'] = nba_df['team_ville'].map({'Toronto Raptors':'Toronto',
     'Boston Celtics':'Boston',
     'Philadelphia 76ers':'Philadelphia',
     'Cleveland Cavaliers':'Cleveland',
     'Indiana Pacers':'Indianapolis',
     'Miami Heat':'Miami–Fort Lauderdale',
     'Milwaukee Bucks':'Milwaukee',
     'Washington Wizards':'Washington, D.C.',
     'Detroit Pistons':'Detroit',
     'Charlotte Hornets':'Charlotte',
     'New York Knicks':'New York City',
     'Brooklyn Nets':'New York City',
     'Chicago Bulls':'Chicago',
     'Orlando Magic':'Orlando',
     'Atlanta Hawks':'Atlanta',
     'Houston Rockets':'Houston',
     'Golden State Warriors':'San Francisco Bay Area',
     'Portland Trail Blazers':'Portland',
     'Oklahoma City Thunder':'Oklahoma City',
     'Utah Jazz':'Salt Lake City',
     'New Orleans Pelicans':'New Orleans',
     'San Antonio Spurs':'San Antonio',
     'Minnesota Timberwolves':'Minneapolis–Saint Paul',
     'Denver Nuggets':'Denver',
     'Los Angeles Clippers':'Los Angeles',
     'Los Angeles Lakers':'Los Angeles',
     'Sacramento Kings':'Sacramento',
     'Dallas Mavericks':'Dallas–Fort Worth',
     'Memphis Grizzlies':'Memphis',
     'Phoenix Suns':'Phoenix'})
    
    df2 = pd.merge(nba_df,cities, left_on= "team_ville", right_on= "Metropolitan area")
    
    df2['W/L%'] = pd.to_numeric(df2['W/L%'])
    df2['W'] = pd.to_numeric(df2['W'])
    df2['L'] = pd.to_numeric(df2['L'])
    df2['Population (2016 est.)[8]'] = pd.to_numeric(df2['Population (2016 est.)[8]'])
    he = ['team','W','L','W/L%','Metropolitan area','Population (2016 est.)[8]']
    df2 = df2[he]
    df2['W/L'] = df2['W']/(df2['L']+df2['W'])
    df2 = df2.groupby('Metropolitan area').mean().reset_index()
    
    return df2

def mlb_correla(): 
    
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    cities.drop([24,25,26,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,48,49,50],0,inplace=True)
    
    mlb_df = mlb_df.head(30)
    
    mlb_df['team_ville'] = mlb_df['team']
    mlb_df['team_ville'] = mlb_df['team_ville'].map({'Boston Red Sox':'Boston',
     'New York Yankees':'New York City',
     'Tampa Bay Rays':'Tampa Bay Area',
     'Toronto Blue Jays':'Toronto',
     'Baltimore Orioles':'Baltimore',
     'Cleveland Indians':'Cleveland',
     'Minnesota Twins':'Minneapolis–Saint Paul',
     'Detroit Tigers':'Detroit',
     'Chicago White Sox':'Chicago',
     'Kansas City Royals':'Kansas City',
     'Houston Astros':'Houston',
     'Oakland Athletics':'San Francisco Bay Area',
     'Seattle Mariners':'Seattle',
     'Los Angeles Angels':'Los Angeles',
     'Texas Rangers':'Dallas–Fort Worth',
     'Atlanta Braves':'Atlanta',
     'Washington Nationals':'Washington, D.C.',
     'Philadelphia Phillies':'Philadelphia',
     'New York Mets':'New York City',
     'Miami Marlins':'Miami–Fort Lauderdale',
     'Milwaukee Brewers':'Milwaukee',
     'Chicago Cubs':'Chicago',
     'St. Louis Cardinals':'St. Louis',
     'Pittsburgh Pirates':'Pittsburgh',
     'Cincinnati Reds':'Cincinnati',
     'Los Angeles Dodgers':'Los Angeles',
     'Colorado Rockies':'Denver',
     'Arizona Diamondbacks':'Phoenix',
     'San Francisco Giants':'San Francisco Bay Area',
     'San Diego Padres':'San Diego'})
    
    df3 = pd.merge(mlb_df,cities, left_on= "team_ville", right_on= "Metropolitan area")
    

    df3['W'] = pd.to_numeric(df3['W'])
    df3['L'] = pd.to_numeric(df3['L'])
    df3['Population (2016 est.)[8]'] = pd.to_numeric(df3['Population (2016 est.)[8]'])
    he = ['team','W','L','Metropolitan area','Population (2016 est.)[8]']
    df3 = df3[he]
    df3['W/L'] = df3['W']/(df3['L']+df3['W'])
    df3 = df3.groupby('Metropolitan area').mean().reset_index()
    
    return df3

def nfl_correla(): 

    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    nfl_df.drop([0,5,10,15,20,25,30,35],0,inplace=True)
    
    cities.drop([13,22,27,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,49,50],0,inplace=True)
    
    l1 = []
    for i in nfl_df['team']:
        i=i.split('*')
        l1.append(i[0])
    nfl_df['team'] = l1
    
    l2 = []
    for i in nfl_df['team']:
        i=i.split('+')
        l2.append(i[0])
    nfl_df['team'] = l2
    
    nfl_df = nfl_df.head(32)
    
    nfl_df['team_ville'] = nfl_df['team']
    nfl_df['team_ville'] = nfl_df['team_ville'].map({'New England Patriots':'Boston',
     'Miami Dolphins':'Miami–Fort Lauderdale',
     'Buffalo Bills':'Buffalo',
     'New York Jets':'New York City',
     'Baltimore Ravens':'Baltimore',
     'Pittsburgh Steelers':'Pittsburgh',
     'Cleveland Browns':'Cleveland',
     'Cincinnati Bengals':'Cincinnati',
     'Houston Texans':'Houston',
     'Indianapolis Colts':'Indianapolis',
     'Tennessee Titans':'Nashville',
     'Jacksonville Jaguars':'Jacksonville',
     'Kansas City Chiefs':'Kansas City',
     'Los Angeles Chargers':'Los Angeles',
     'Denver Broncos':'Denver',
     'Oakland Raiders':'San Francisco Bay Area',
     'Dallas Cowboys':'Dallas–Fort Worth',
     'Philadelphia Eagles':'Philadelphia',
     'Washington Redskins':'Washington, D.C.',
     'New York Giants':'New York City',
     'Chicago Bears':'Chicago',
     'Minnesota Vikings':'Minneapolis–Saint Paul',
     'Green Bay Packers':'Green Bay',
     'Detroit Lions':'Detroit',
     'New Orleans Saints':'New Orleans',
     'Carolina Panthers':'Charlotte',
     'Atlanta Falcons':'Atlanta',
     'Tampa Bay Buccaneers':'Tampa Bay Area',
     'Los Angeles Rams':'Los Angeles',
     'Seattle Seahawks':'Seattle',
     'San Francisco 49ers':'San Francisco Bay Area',
     'Arizona Cardinals':'Phoenix'}) 
    
    df4 = pd.merge(nfl_df,cities, left_on= "team_ville", right_on= "Metropolitan area")
    
    
    df4['W'] = pd.to_numeric(df4['W'])
    df4['L'] = pd.to_numeric(df4['L'])
    df4['Population (2016 est.)[8]'] = pd.to_numeric(df4['Population (2016 est.)[8]'])
    he = ['team','W','L','Metropolitan area','Population (2016 est.)[8]']
    df4 = df4[he]
    df4['W/L'] = df4['W']/(df4['L']+df4['W'])
    df4 = df4.groupby('Metropolitan area').mean().reset_index()
    
    return df4





def sports_team_performance():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    nfl = nfl_correla()
    nba = nba_correla()
    mlb = mlb_correla()
    nhl = nhl_correla()

    nba_nfl = pd.merge(nba,nfl, on='Metropolitan area')
    pval_nba_nfl = stats.ttest_rel(nba_nfl['W/L_x'],nba_nfl['W/L_y'])[1]
    nba_nhl = pd.merge(nba,nhl, on='Metropolitan area')
    pval_nba_nhl = stats.ttest_rel(nba_nhl['W/L_x'],nba_nhl['W/L_y'])[1]
    mlb_nfl = pd.merge(mlb,nfl, on='Metropolitan area')
    pval_mlb_nfl = stats.ttest_rel(mlb_nfl['W/L_x'],mlb_nfl['W/L_y'])[1]
    mlb_nhl = pd.merge(mlb,nhl, on='Metropolitan area')
    pval_mlb_nhl = stats.ttest_rel(mlb_nhl['W/L_x'],mlb_nhl['W/L_y'])[1]
    mlb_nba = pd.merge(mlb,nba, on='Metropolitan area')
    pval_mlb_nba = stats.ttest_rel(mlb_nba['W/L_x'],mlb_nba['W/L_y'])[1]
    nhl_nfl = pd.merge(nhl,nfl, on='Metropolitan area')
    pval_nhl_nfl = stats.ttest_rel(nhl_nfl['W/L_x'],nhl_nfl['W/L_y'])[1]
    
    pv = {'NFL': {"NFL": np.nan, 'NBA': pval_nba_nfl, 'NHL': pval_nhl_nfl, 'MLB': pval_mlb_nfl},
       'NBA': {"NFL": pval_nba_nfl, 'NBA': np.nan, 'NHL': pval_nba_nhl, 'MLB': pval_mlb_nba},
       'NHL': {"NFL": pval_nhl_nfl, 'NBA': pval_nba_nhl, 'NHL': np.nan, 'MLB': pval_mlb_nhl},
       'MLB': {"NFL": pval_mlb_nfl, 'NBA': pval_mlb_nba, 'NHL': pval_mlb_nhl, 'MLB': np.nan}
      }

    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    #sports = ['NFL', 'NBA', 'NHL', 'MLB']
    #p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    p_values = pd.DataFrame(pv)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

sports_team_performance()

C:\Users\Pierre\AppData\Local\Temp\ipykernel_14556\504393482.py:230: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  nfl_df.drop([0,5,10,15,20,25,30,35],0,inplace=True)
C:\Users\Pierre\AppData\Local\Temp\ipykernel_14556\504393482.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  cities.drop([13,22,27,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,49,50],0,inplace=True)
C:\Users\Pierre\AppData\Local\Temp\ipykernel_14556\504393482.py:95: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  cities.drop([16,17,19,20,21,22,23,26,29,30,31,34,35,36,37,39,40,43,44,47,48,49,50],0,inplace=True)
C:\Users\Pierre\AppData\Local\Temp\ipykernel_14556\504393482.py:173: FutureWarning: In a future version of pandas all arguments of DataFrame.drop

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
